<a href="https://colab.research.google.com/github/hagapova/slam/blob/master/%D0%9E%D1%82%D1%87%D0%B5%D1%82_%D0%BF%D0%BE_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B5_2_%D0%BA%D1%83%D1%80%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Abstract

Получив результаты работы, проделанной нами к концу марта 2019, мы встретились со следующей проблемой: траектория, восстановленная по 2D-изображениям (раскадровка видео, на котором запечатлен поворот), имела такой вид:

![alt text](https://i.ibb.co/NVppSZw/Screen-Shot-2019-08-08-at-12-25-20-PM.png)

Наклонная прямая показалась нам похожа на приближенный поворот направо. Но движение траектории “вперед-назад” нельзя было объяснить никак, кроме как ошибкой в реализации алгоритма. Исправление этой ошибки и улучшение работы нашего одометра мы перед собой и поставили целью в данной работе.

В ходе выявления ошибок в коде мы пришли к следующим выводам и результатам (подробно о каждом из которых мы расскажем ниже):
1. Мы неправильно считали координаты положений камеры. 
1. Мы неправильно понимали принцип работы одной из функций, которые мы использовали для вычисления существенной матрицы. В этой связи мы упустили в нашей работе один важный шаг -- отслеживание объектов.
2. По существенной матрице можно восстановить вектор перемещения и две матрицы поворота -- мы узнали, как именно выбор знака вектора и одной из матриц влияет на изменение траектории и как выбрать правильный вариант из четырех возможных (в марте мы делали это по принципу выбора ближайшего к предыдущему, что было неверной стратегией).
3. Мы познакомились с понятием “dominant direction” и ввели в наш алгоритм соответствующую проверку.


Рассмотрим отдельно каждый пункт.


## Как мы считали положения камеры, и как нужно на самом деле?

В весенней работе наше предположение было простым: каждое следующее положение определяется поворотом на матрицу поворота и смещением на вектор перемещения:

$$(x_{i+1}, y_{i+1}, z_{i+1})^{T} = R_{i} \times  (x_{i}, y_{i}, z_{i})^{T} + t_{i}$$

Мы не учли, что, вообще говоря, матрица поворота и вектор перемещения вычисляются без знания о том, где находится камера и как она была повернута относительно начала своего движения. Значит, матрица и вектор выражены в локальной системе координат, и для каждого следующего перемещения она своя. Нам нужно сперва пересчитать вектор и матрицу в глобальной системе координат, в которой мы и будем вычислять траекторию.

Из линейной алгебры мы знаем, что матрица поворота $R(({X}_{1}, {Y}_{1}, {Z}_{1}), ({X}_{2}, {Y}_{2}, {Z}_{2}))$ -- это как раз матрица перехода из $ ({X}_{1}, {Y}_{1}, {Z}_{1})$ в $({X}_{2}, {Y}_{2}, {Z}_{2})$. 

Кроме того, если мы имеем два последовательных перехода из системы $({X}_{1}, {Y}_{1}, {Z}_{1})$ в систему $({X}_{2}, {Y}_{2}, {Z}_{2})$, и из $({X}_{2}, {Y}_{2}, {Z}_{2})$ в $({X}_{3}, {Y}_{3}, {Z}_{3})$, то:
$$C(({X}_{1}, {Y}_{1}, {Z}_{1}), ({X}_{2}, {Y}_{2}, {Z}_{2})) =\\
C(({X}_{1}, {Y}_{1}, {Z}_{1}), ({X}_{2}, {Y}_{2}, {Z}_{2}))\times C(({X}_{2}, {Y}_{2}, {Z}_{2}),({X}_{3}, {Y}_{3}, {Z}_{3}))$$

Таким образом, матрица перехода из системы координат $i$-го положения в глобальную систему координат -- это последовательное произведение матриц поворота первых $i-1$ шагов.

Теперь мы будем считать положения так:

$$i = 0 \Rightarrow C =
\begin{pmatrix} 
1 & 0 & 0 \\
0 & 1 & 0\\
0 & 0 & 1
\end{pmatrix}$$

$$
C = C \times R_{i} \\
t^{global}_{i} = C^{-1}\times t_{i}\\
(x_{i+1}, y_{i+1}, z_{i+1})^{T} = (x_{i}, y_{i}, z_{i})^{T} + t^{global}_{i}\\
$$

##Что мы не понимали про cv.findEssentialMat(...)?

При работе с функцией, считающей существенную матрицу, мы упустили важный факт: принимая на вход два массива значимых точек с пары картинок, алгоритм предполагает, что эти массивы будут одного размера, а i-й элемент первого массива будет соответствовать i-му элементу второго, то есть эти две точки будут соответствовать одной и той же точке в 3D-пространстве. По сути, для того, чтобы предоставить функции два массива соответствующих точек, мы должны были выполнить такую фундаментальную задачу, как Object Tracking, то есть предположить для каждой точки с первого изображения, где соответствующая ей точка окажется на второй. Этот шаг в нашей весенней работе был пропущен.

Исправляя ошибку, мы вручную стали решать задачу Object Tracking, отслеживая углы описанных вокруг одних и тех же объектов прямоугольников (наши значимые точки). Для большей устойчивости мы добавили к точкам, соответствующим каждому объекту, кроме углов, еще середины сторон описанного прямоугольника и его центр. Это связано с нашим предположением, сделанным в конце весенней работы, о том, что значимых точек нужно использовать больше.

Итак, одному объекту у нас соответствует девять значимых точек. Допустим, мы имеем пару последовательных кадров -- А и Б. На каждом из них распознан объект, о котором мы знаем только его тип, название неподвижного объекта (допустим, это “дерево”). Дальше будем считать, что мы уже знаем, что пара объектов с кадров А и Б принадлежит одному типу (мы включили в наш код соответствующую проверку). 

Перед нами стоят задачи:
1. Понять: дерево с кадра А соответствует тому же реальному дереву, что и дерево с кадра Б? Если да, то:
2. Какая из 9 значимых точек дерева с кадра А соответствует какой из точек дерева с кадра Б? 

Если мы ответим на вопрос 2, то получим два массива, в которых i-е элементы -- соответствующие друг другу значимые точки.
Для каждого объекта, распознанного на кадре А, мы найдем пару на кадре Б, либо поймем, что на кадре Б такой пары не найдется (ведь точка, которая видна на кадре А, на Б уже могла исчезнуть из поля зрения).

Тогда, задав минимальное приемлемое количество таких соответствий, мы можем считать, что соответствующие точки с кадра Б -- это и есть предположительные координаты точек с кадра А (ведь смысл у них одинаковый -- угол/центр/середина стороны описанного прямоугольника одного и того же 3D-объекта). А значит, с таким грубым предположением, мы выполним задачу Object Tracking. 

На практике мы выяснили, что для нашего датасета и нашей нейронной сети в качестве решения вопроса 1 подходило посчитать отношение пересечения описанных прямоугольников к их объединению (назовем эту величину IoU двух объектов). Если на кадре А распознано дерево 1, а на кадре Б распознано дерево 2, то мы считаем деревья 1 и 2 одним и тем же деревом только в следущем случае: на кадре Б не распознано другого дерева, для которого IoU с деревом 1 было бы больше, чем IoU деревьев 1 и 2.

Получив ответ на вопрос 1 для каждого объекта с первого из двух последовательных кадров, мы сопоставляем углы, середины сторон и центры описанных прямоугольников, и получаем ответ на вопрос 2, а значит -- два подходящих для функции ```findEssentialMat``` массива точек.

Так мы проходим по всем последовательным парам кадров, но пропускаем кадры, на которых распознано меньше двух общих с предыдущим кадром объектов (то есть не больше 9 пар точек). Так мы не попадем ни в ситуацию, когда значимых точек будет недостаточно для достоверного вычисления существенной матрицы, ни в ситуацию, когда на кадрах просто не распозналось общих объектов, общих точек не будет вообще, и считать будет нечего.


##Как выбрать -- $R_{1}$ или $R_{2}$, $t$ или $-t$?

В весенней работе мы упомянули две функции -- ```decomposeEssentialMat``` и ```recoverPose```. 

И та, и другая принимают на вход существенную матрицу и вычисляют из нее матрицу поворота и вектор перемещения при помощи SVD-разложения. В ходе SVD из-за свойств существенной матрицы возникает двойственность -- центральный из полученных множителей может быть равен как $ W$, так и ${W}^{Т} $, где $$W = \begin{pmatrix} 
0 & 1 & 0 \\
-1 & 0 & 0\\
0 & 0 & -1
\end{pmatrix}$$
Подробное объяснение этого факта со всеми алгебраическими выкладками описано в публикации [[1]](http://igt.ip.uca.fr/~ab/Classes/DIKU-3DCV2/Handouts/Lecture16.pdf) (авторы -- Adrien Bartoli, CNRS – LASMEA, Клермон-Ферран, и Søren I. Olsen, DIKU, Копенгаген).

Для нас же важен следующий факт: когда на выходе ```decomposeEssentialMat``` предлагает две возможных матрицы поворота ($R_{1}$ и $R_{2}$) и вектор перемещения ($t$), мы имеем дело именно с описанной выше двойственностью. 
Как мы уже знаем, два возможных знака вектора перемещения и две возможных матрицы поворота дают нам четыре возможных варианта изменения позиции камеры. Теперь, разобравшись, чем именно отличаются две предложенных матрицы, мы знаем, как именно выбор одного из четырех вариантов повлияет на строение траектории:
1. Выбор матрицы отвечает за отражение траектории относительно линии, соединяющей координаты камер (в нашем случае, предположение о том, где будет камера, отражается от линии, соединяющей предыдущие два положения камеры).
2. Выбор знака вектора отвечает за направление движения: вперед или назад.

В результате сопоставления точек (то есть устранения предыдущей ошибки), мы получили такую траекторию:
![alt text](https://i.ibb.co/vdzFDYd/Screen-Shot-2019-07-26-at-5-13-38-PM.png)


Здесь значимые точки сопоставлены правильно, но выбор матрицы поворота мы все еще делаем по принципу ближайшей к предыдущей матрице, то есть, почти случайно, а вектор перемещения мы вообще принимаем как есть -- весной мы предполагали, что выбор знака за нас уже сделала функция ```decomposeEssentialMat```. 

На самом же деле, если мы используем функцию ```decomposeEssentialMat```, то и выбор матрицы, и выбор знака вектора лежит на нас, тогда как ```recoverPose``` делает выбор при помощи RANSAC. В предыдущей работе мы не придавали значения важности этого выбора, а поэтому сочли функцию ```recoverPose``` аналогичной действию функции ```decomposeEssentialMat``` вместе с выбором правильного варианта вручную. Теперь мы знаем, что это неверно, и поэтому заменяем использование ```decomposeEssentialMat``` на ```recoverPose```.

По ходу мы снова пропускаем кадры, если ```recoverPose``` нашла среди точек слишком много аутлаеров (это один из параметров, который возвращает функция, мы поставили границу -- не больше 3 аутлаеров из 18), и поэтому ее решение будет ненадежным.


## Что такое dominant direction?
Теперь мы получили такую траекторию:
![alt text](https://i.ibb.co/JyxpKS6/Screen-Shot-2019-07-30-at-5-59-53-PM.png)
Эта фигура больше всего похожа на прямую линию, а прямая линия -- промежуточный результат, которого мы хотели добиться на данном этапе.
Здесь зеленая точка -- это точка старта камеры, а красная -- точка финиша. 

Мы видим, что кроме двух ошибок в выборе вектора перемещения (места, где траектория двигается назад-вперед, более жирные, чем остальные), траектория целиком пошла назад. Но мы уже знаем, почему: позиции мы считаем с учетом предыдущих положений, и если один раз вначале мы ошиблись и получили отрицательное направление, то эта ошибка развернет всю последующую траекторию.

В статье [[2]](https://avisingh599.github.io/vision/monocular-vo/), где автор решает задачу, схожую с нашей, упоминается понятие "dominant motion". Суть его такова: автомобиль с камерой (это как раз наш случай) физически не может резко сдвинуться назад. Любой физически возможный поворот будет состоять из перемещений в полуплоскости перед камерой (видимо, автор игнорирует возможность заднего хода, и здесь мы последуем его примеру). Тогда, если алгоритм решает, что камера резко сдвинулась назад, то есть переместилась в точку в полуплоскости позади себя, то такое движение мы просто игнорируем, считая, что это ошибка.

Введя простую проверку на координату $X$ каждого положения, мы убеждаемся, что камера не двигается резко назад. В противном случае мы не игнорируем это перемещение, а "разворачиваем" -- то есть учитываем соответствующий вектор перемещения с минусом. Это связано с тем, что мы уже знаем о характере недочетов нашей реализации -- ошибка где-то в выборе матрицы поворота и векторе перемещения.

Так мы получили такую траекторию (на большем количестве кадров):
![alt text](...)

В введенной нами проверке есть свои минусы -- мы меняем только знак вектора, а для точного воспроизведения траектории нам нужно проверять и матрицу поворота.

## Что еще?

На всех полученных графиках мы видим наклон. Не разобравшись в проблеме, мы предполагали, что наклонная прямая -- это "примерно" поворот направо. Теперь, когда в поиске ошибки мы проверяли каждый шаг отдельно, мы понимаем, что это не так -- ведь в каждом перемещении не содержится никакой информации о том, что дальше будет резкий поворот. Это (а так же линейное возрастание координаты $Z$, которая должна вообще оставаться константно равной нулю) наводит нас на мысль, что глобальная система координат смещена, и нужно развернуть ее обратно. 

##Литература
[[1]](http://igt.ip.uca.fr/~ab/Classes/DIKU-3DCV2/Handouts/Lecture16.pdf) Camera Motion From Two-View Relationships (Section 9.6), Adrien Bartoli, CNRS – LASMEA, Clermont-Ferrand, France, Søren I. Olsen, DIKU, Copenhagen, Denmark

[[2]](https://avisingh599.github.io/vision/monocular-vo/), Monocular Visual Odometry using OpenCV, Avi Singh